In [1]:
import json
import pandas as pd
from pathlib import PurePath

In [1]:
def filename(x):
#     print(x)
    # return x
    if not hasattr(x, '__len__') or len(x) == 0:
        return "''"
    pure_path = PurePath(x[0])
    # print(pure_path.parent.name)
    if pure_path.parent.name:
        return f"../{pure_path.name}"
    return pure_path.name


def llc_to_df(json_file, name, index_column, per_column):
    with open(json_file) as file:
        json_arr = json.load(file)
    df = pd.json_normalize(json_arr[-1])
    df["all.agg_bw"] = df["all.agg_bw"].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".3f") + "GB/s")
    df["all.agg_size"] = df["all.agg_size"].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".3f") + "GB")
    df['all.unique_filename'] = df["all.unique_filename"].apply(filename)
    df['file'] = df['all.unique_filename']
    df['rank'] = df['all.unique_rank']
    df['start_seconds'] = df['start'].apply(lambda x: format(float(x)/1e7, ".2f"))
    df['stop_seconds'] = df['stop'].apply(lambda x: format(float(x)/1e7, ".2f"))
    df['time'] = df['start_seconds'] + '-' + df['stop_seconds']
    df['time_start'] = df['time_start'].apply(lambda x: format(x, ".2f"))
    df['time_end'] = df['time_end'].apply(lambda x: format(x, ".2f"))
    df["md_io_ratio"] = df["md_io_ratio"].apply(lambda x: format(float(x)*100.0, ".2f") + "%")
    df[per_column] = df[per_column].apply(lambda x: format(float(x)*100.0, ".2f") + "%")    
    df.drop(columns=['all.unique_filename', 'all.unique_func_id', 'all.unique_rank'], inplace=True)
    df.drop(columns=['is_collective', 'is_hdf5', 'is_md_only', 'start', 'stop', 'start_seconds', 'stop_seconds'], inplace=True)
    if index_column == 'file':
        df.drop(columns=['rank', 'time'], inplace=True)
    elif index_column == 'rank':
        df.drop(columns=['file', 'time'], inplace=True)
    else:
        df.drop(columns=['file', 'rank', 'time_start', 'time_end'], inplace=True)
    df.rename(columns={'all.total_index': 'all.ops'}, inplace=True)
    df.rename(columns={'all.total_unique_filename': 'all.files'}, inplace=True)
    df.rename(columns={'all.total_unique_func_id': 'all.funcs'}, inplace=True)
    df.rename(columns={'all.total_unique_rank': 'all.ranks'}, inplace=True)
    df.rename(columns={'all.xfer_size_fmt': 'all.xfer_size'}, inplace=True)
    df.rename(columns={'read.xfer_size_fmt': 'r.xfer'}, inplace=True)
    df.rename(columns={'write.xfer_size_fmt': 'w.xfer'}, inplace=True)
    df.rename(columns={per_column: '%'}, inplace=True)
    df = df.loc[:, ~df.columns.str.startswith('per_')]
    df = df.loc[:, ~df.columns.str.startswith('metadata.')]
    df = df.loc[:, ~df.columns.str.startswith('read.')]
    df = df.loc[:, ~df.columns.str.startswith('write.')]
    df.insert(0, index_column, df.pop(index_column))
    df.insert(1, '%', df.pop('%'))
    if index_column != 'time':
        df.insert(2, 'time_start', df.pop('time_start'))
        df.insert(3, 'time_end', df.pop('time_end'))
    tuple_cols = df.columns.str.split('.')
    tuples = [tuple(i) if len(i) == 2 else tuple([name, i[0]]) for i in tuple_cols]
    df.columns = pd.MultiIndex.from_tuples(tuples)
    return df
